In [17]:
import pandas as pd

train_df_csv = pd.read_csv('../data/train_processed.csv',delimiter = ';')
test_df_csv = pd.read_csv('../data/test_processed.csv',delimiter = ';')

Открываем заранее обработанные массивы и начинаем ориентироваться на лабораторную работу №2 по МОМО.

Интересно, что некоторые из строк этого файла представляют из себя NaN, что показывает, что где-то ранее была допущена ошибка, которую стоит исправить.
Но сейчас мы будем заниматься приведением данных в надлежащий вид.

In [18]:
train_df_csv

,request_ts,user_id,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,browser_version,os,os_version,target
0,1701011363,fb858e8e0a2bec074450eaf94b627fd3,c31b4e,470e75,f6155e,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,0
1,1700986581,46a5f128fd569c764a92c2eaa788095e,c31b4e,44520b,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,111.0.0,Android,10,0
2,1701011071,5a74e9ac53ffb21a20cce117c0ad77ba,c31b4e,616bb9,af47f1,12498,2451,10304,-6380,11608,3106,-2188,10573,3347,21870,Yandex Browser,20.12.5,Android,11,0
3,1700992803,af735816ca19115431ae3d89518c8c91,c31b4e,3c9dca,e56e80,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Chrome Mobile,119.0.0,Android,10,0
4,1701021666,364f0ae0a3f29a685c4fb5bae6033b9a,c31b4e,776e76,10b7947,11731,4045,22213,-1184,-8992,9381,-3496,-3120,-899,16817,Yandex Browser,18.11.1,Android,4.4.4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
601285,1701009148,11516096d9f97463424523e6154d5255,c31b4e,8ccc01,e56e80,18174,17591,3257,2581,8043,13263,2312,1791,9121,5170,Yandex Browser,23.11.0,Android,13,1
601286,1700954940,d078ecb7c7e9eaf65189fb3d1bed7871,c31b4e,52b73e,e56e80,14961,13188,3978,1812,13751,14055,1141,-10698,7083,9270,Chrome,119.0.0,Windows,10,0
601287,1700994910,3543d64627ead3a519e3199834e2a148,c31b4e,33ed7a,10b7947,12332,17481,12664,-984,5146,11562,4316,15017,1423,-981,Chrome Mobile,105.0.0,Android,11,0
601288,1700989030,37df5ff1d739f61d442b164db6281e46,c31b4e,97023a,e56e80,13459,-6413,13160,13187,5498,3875,13246,10259,-1630,-14647,Chrome Mobile,94.0.4606,Android,11,0


Оценим количество людей разного гендера

In [19]:
train_df_csv['target'].value_counts()

target
0    313572
1    287718
Name: count, dtype: int64

Единственной численной характеристикой будет request_ts, остальные я пока буду считать не численными, а категориальными

In [20]:
numeric_features = [
    'request_ts',
    'component0',
    'component1',
    'component2',
    'component3',
    'component4',
    'component5',
    'component6',
    'component7',
    'component8',
    'component9'
    ]  #потом сюда попадёт год выпуска браузера и год выпуска ОС
categorial_features=[
     'user_id',
 'country_id',
 'region_id',
 'timezone_id',
 'browser',
 'browser_version',
 'os',
 'os_version',
 'target']

categorial_features

['user_id',
 'country_id',
 'region_id',
 'timezone_id',
 'browser',
 'browser_version',
 'os',
 'os_version',
 'target']

Теперь у нас будет разный подход к числовым и категориальным.

In [21]:
from sklearn.pipeline import Pipeline # для создания pipeline
from sklearn.compose import ColumnTransformer # для преобразования колонок
from sklearn.preprocessing import StandardScaler

import numpy as np

from sklearn.base import BaseEstimator, TransformerMixin 
from sklearn.pipeline import Pipeline  
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import PowerTransformer 
from sklearn.preprocessing import OneHotEncoder 
from sklearn.compose import ColumnTransformer  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import LabelEncoder  
from sklearn.linear_model import LogisticRegression  
from sklearn.metrics import f1_score  
from sklearn.metrics import classification_report 
from sklearn.model_selection import cross_validate  
from sklearn.model_selection import StratifiedKFold

#TODO поудалять, что не прегиодилось

Это мы забираем с лабораторной работы, не зря же мы это тогда писали. Оно умеет захватывать в одну категорию редкое. Так что условные два часовых пояса, где есть по 2 пользователя можно объединить в один. И может оказаться, что  в тех странных часовых поясах тусуются моряки, которые чаще мужчины. И так же со странами. Может оказаться, что все ребята, которые находятся в редких странах мужчины.

In [22]:
class QuantileReplacer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05):
        self.threshold = threshold
        self.quantiles = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include='number'):
            low_quantile = X[col].quantile(self.threshold)
            high_quantile = X[col].quantile(1 - self.threshold)
            self.quantiles[col] = (low_quantile, high_quantile)
        return self

    def transform(self, X):
        X_copy = X.copy()
        for col in X.select_dtypes(include='number'):
            low_quantile, high_quantile = self.quantiles[col]
            rare_mask = ((X[col] < low_quantile) | (X[col] > high_quantile))
            if rare_mask.any():
                rare_values = X_copy.loc[rare_mask, col]
                replace_value = np.mean([low_quantile, high_quantile])
                if rare_values.mean() > replace_value:
                    X_copy.loc[rare_mask, col] = high_quantile
                else:
                    X_copy.loc[rare_mask, col] = low_quantile
        return X_copy
    
class RareGrouper(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0.05, other_value='Other'):
        self.threshold = threshold
        self.other_value = other_value
        self.freq_dict = {}

    def fit(self, X, y=None):
        for col in X.select_dtypes(include=['object']):
            freq = X[col].value_counts(normalize=True)
            self.freq_dict[col] = freq[freq >= self.threshold].index.tolist()
        return self

    def transform(self, X, y=None):
        X_copy = X.copy()
        for col in X.select_dtypes(include=['object']):
            X_copy[col] = X_copy[col].apply(lambda x: x if x in self.freq_dict[col] else self.other_value)
        return X_copy

Вот это мы выведем исключительно для того, чтобы копировать столбики и не листать по 4 экрана.

In [23]:
train_df_csv.columns

Index(['request_ts', 'user_id', 'country_id', 'region_id', 'timezone_id',
       'component0', 'component1', 'component2', 'component3', 'component4',
       'component5', 'component6', 'component7', 'component8', 'component9',
       'browser', 'browser_version', 'os', 'os_version', 'target'],
      dtype='object')

Начинается хорошее.

Отмечу, что мы должны использовать StandardScaler если данные имеют нормальное распределение и PowerTransformer если нет. 
Предположим, что request_ts имеет нормальное распределение. (спорно, но пока опустим)

In [24]:
num_pipe_request_ts = Pipeline([
    ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_request_ts = ['request_ts']

cat_pipe_country_id = Pipeline([
    ('replace_rare', RareGrouper(threshold=0.001, other_value='Other')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])
cat_country_id = ['country_id']

cat_pipe_region_id = Pipeline([
    ('replace_rare', RareGrouper(threshold=0.001, other_value='Other')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])
cat_region_id = ['region_id']

cat_pipe_timezone_id = Pipeline([
    ('replace_rare', RareGrouper(threshold=0.001, other_value='Other')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])
cat_timezone_id = ['timezone_id']

num_pipe_component0 = Pipeline([
    ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component0 = ['component0']

num_pipe_component1 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component1 = ['component1']

num_pipe_component2 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component2 = ['component2']

num_pipe_component3 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component3 = ['component3']

num_pipe_component4 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component4 = ['component4']

num_pipe_component5 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component5 = ['component5']

num_pipe_component6 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component6 = ['component6']

num_pipe_component7 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component7 = ['component7']

num_pipe_component8 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component8 = ['component8']

num_pipe_component9 = Pipeline([
     ('QuantReplace', QuantileReplacer(threshold=0.01, )),
    ('scaler', StandardScaler())
])
num_component9 = ['component9']

cat_pipe_browser = Pipeline([
    ('replace_rare', RareGrouper(threshold=0.001, other_value='Other')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])
cat_browser = ['browser']

cat_pipe_os = Pipeline([
    ('replace_rare', RareGrouper(threshold=0.001, other_value='Other')),
    ('encoder', OneHotEncoder(drop='if_binary', handle_unknown='ignore', sparse_output=False))
])
cat_os = ['os']

Теперь у нас есть возможность создать общий пайплайн

'request_ts', 'user_id', 'country_id', 'region_id', 'timezone_id',
       'component0', 'component1', 'component2', 'component3', 'component4',
       'component5', 'component6', 'component7', 'component8', 'component9',
       'browser', 'browser_version', 'os', 'os_version', 'target'

In [25]:
preprocessors_all = ColumnTransformer(transformers=[
    ('num_request_ts', num_pipe_request_ts, num_request_ts),
    #('cat_user_id', cat_pipe_user_id, cat_user_id), не уверен, что это осмысленно
    ('cat_country_id', cat_pipe_country_id, cat_country_id), 
    ('cat_region_id', cat_pipe_region_id, cat_region_id), 
    ('cat_timezone_id', cat_pipe_timezone_id, cat_timezone_id), 
    ('num_component0', num_pipe_component0, num_component0),
    ('num_component1', num_pipe_component1, num_component1), 
    ('num_component2', num_pipe_component2, num_component2), 
    ('num_component3', num_pipe_component3, num_component3), 
    ('num_component4', num_pipe_component4, num_component4), 
    ('num_component5', num_pipe_component5, num_component5), 
    ('num_component6', num_pipe_component6, num_component6), 
    ('num_component7', num_pipe_component7, num_component7), 
    ('num_component8', num_pipe_component8, num_component8), 
    ('num_component9', num_pipe_component9, num_component9), 
    ('cat_browser', cat_pipe_browser, cat_browser), 
    #('cat_os', cat_pipe_os, cat_os)
])

Далле мы отделяем целевую переменную от тех, которые будут использоваться для угадывания.

Поля с версиями роняем чисто на всякий случай. Они у нас никак не обрабатываются так или иначе. (пока что)

In [26]:
X, y = train_df_csv.drop(columns = ['target','browser_version', 'os_version']), train_df_csv['target']

## Разбиение на тестовую и валидационную выборки
Стоит отметить, что мы получаем тестовую выборку из файла train, что может вызвать разночтения. Поясняем, это нормальный процесс, так как файл test не содержит правильных ответов, а нам хочется провести какой-то тест внутренний, а не сдав это преподавателю, мы будем проверять качество на 15% данных, которые не учавствовали в процессе обучения. Так что мы ставим test_size = 0.15

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X, y,   test_size=0.15, random_state=42)

In [28]:
X_train

,request_ts,user_id,country_id,region_id,timezone_id,component0,component1,component2,component3,component4,component5,component6,component7,component8,component9,browser,os
385201,1700987756,4964aaadbe0004cddb414859d6787d0e,c31b4e,895e40,109a080,2612,5571,23275,8842,-6027,-1992,-18720,-98,-11025,8009,Yandex Browser,Android
483415,1701009688,282fe5ccb67ca74d0487fac4a1db33b4,121db33,2d7eb6,ac9429,-3578,2164,-12993,5330,5018,8603,-2646,8588,-30424,-4684,Chrome Mobile WebView,Android
551633,1700946994,c4428e791d2076a1d74c0a9d8474556d,c31b4e,7f70c5,e56e80,11052,14907,8913,-7670,16619,1797,-6249,12806,-245,10449,Yandex Browser,Windows
328803,1701018535,0aec8954714f9783d5913838fa977970,c31b4e,52b73e,e56e80,14968,15383,7747,7381,17135,8197,3645,2962,5552,-6485,Chrome Mobile,Android
265436,1700988515,e052da645c9ae62ed0271d1c6a78e865,c31b4e,8bd341,e56e80,7670,5426,11701,-10557,16244,3785,-10,10452,6456,17047,Yandex Browser,Android
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110268,1700954210,72d4034045125de32870b49c67355ccd,e48e27,NaN,f0c3df,3420,9278,11069,17604,8500,-4095,8092,9831,8240,12533,Yandex Browser,Android
259178,1700998697,8d8c88ea1d38a78b5b349f326fbdba2a,1234f1d,781fdd,1054b86,6199,19698,-1199,9735,7563,10296,12285,11282,6463,1541,Chrome Mobile WebView,Android
365838,1701023589,073221668d6b7aba54f93661237a76c1,c31b4e,8ccc01,e56e80,20518,17547,5040,491,7548,8339,6363,4367,8468,-5307,Chrome Mobile,Android
131932,1700983505,dee4dcf2cd3553ef071791723b5637e6,c31b4e,97023a,e56e80,4608,6414,12033,-293,20533,10824,4382,-10065,11514,7684,Edge,Windows


In [29]:
X_train_prep = preprocessors_all.fit_transform(X_train)
print(str(len(X_train_prep)) +"x"+str(len(X_train_prep[0])))

C:\Users\geniuslat\AppData\Local\Temp\ipykernel_18024\2567078331.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1700947095.95' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_copy.loc[rare_mask, col] = low_quantile
C:\Users\geniuslat\AppData\Local\Temp\ipykernel_18024\2567078331.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-16075.150000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_copy.loc[rare_mask, col] = low_quantile


511096x207


In [30]:
X_val_prep = preprocessors_all.transform(X_val)


C:\Users\geniuslat\AppData\Local\Temp\ipykernel_18024\2567078331.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '-16075.150000000001' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  X_copy.loc[rare_mask, col] = low_quantile


In [31]:
model = LogisticRegression(random_state = 42)


In [32]:
model.fit(X_train_prep, y_train)

LogisticRegression(random_state=42)

Напишем функцию для применения нашей обученной модели

In [34]:
def GetPrediction(model, X, y, metric = f1_score, **kwargs):
    
    prediction = model.predict(X)
    return metric(y, prediction, **kwargs)

Оценим качество модели

In [35]:
f" f1 на тренировочной выборке: {GetPrediction(model, X_train_prep, y_train):.2f}" +'\n'+f" f1 на валидационной выборке: {GetPrediction(model, X_val_prep, y_val):.2f}"

' f1 на тренировочной выборке: 0.67\n f1 на валидационной выборке: 0.66'